## Gathering Electricity Prices in USA

Load libraries

In [1]:
library(jsonlite)
library(tidyverse)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


In [2]:
options(repr.matrix.max.rows=20)

Read the list of United States Zip Codes
- The data file is downloaded from <a href="https://www.aggdata.com/download_sample.php?file=us_postal_codes.csv">AggData</a>

In [3]:
cities.usa = read_csv("data/us_postal_codes.csv")
head(cities.usa)

Parsed with column specification:
cols(
  `Zip Code` = col_integer(),
  `Place Name` = col_character(),
  State = col_character(),
  `State Abbreviation` = col_character(),
  County = col_character(),
  Latitude = col_double(),
  Longitude = col_double()
)


Zip Code,Place Name,State,State Abbreviation,County,Latitude,Longitude
501,Holtsville,New York,NY,Suffolk,40.8154,-73.0451
544,Holtsville,New York,NY,Suffolk,40.8154,-73.0451
1001,Agawam,Massachusetts,MA,Hampden,42.0702,-72.6227
1002,Amherst,Massachusetts,MA,Hampshire,42.3671,-72.4646
1003,Amherst,Massachusetts,MA,Hampshire,42.3919,-72.5248
1004,Amherst,Massachusetts,MA,Hampshire,42.3845,-72.5132


We care about Idaho and Washington

In [4]:
cities.selected = cities.usa %>%
    filter(`State Abbreviation` == 'OR') %>%
    select(`Place Name`, State) %>%
    distinct()
cities.selected

Place Name,State
Antelope,Oregon
Aurora,Oregon
Beaverton,Oregon
Beavercreek,Oregon
Boring,Oregon
Bridal Veil,Oregon
Brightwood,Oregon
Canby,Oregon
Cascade Locks,Oregon
Clackamas,Oregon


How many cities do we need to retrieve?

In [5]:
cities.selected %>%
    nrow()

[1] 373

Create a place list

In [6]:
places = cities.selected %>%
    mutate(`Place Name` = gsub(" ", "%20", `Place Name`)) %>%
    .$`Place Name`

In [7]:
length(places)

[1] 373

A function to retrieve electricity prices.
- This function uses <a href = "https://developer.nrel.gov/docs/electricity/utility-rates-v3/#request-url">NREL API</a>.
- Link Example: "https&#58;//developer.nrel.gov/api/utility_rates/v3.json?api_key=yourKey&address=boise&lat=20&lon=20"

In [8]:
get_electricity_prices = function(placesList, maxCallPerCity=2, callInterval=10) {
    
    totalCityNum = length(placesList)
    currentCityNum = 1
    
    for (city in placesList) {
        linkAddress = sprintf("https://developer.nrel.gov/api/utility_rates/v3.json?api_key=qtSpExfVW4iaoxskxPOXVrUEiZQcj7RFaVayvJ3H&address=%s", city)
        message("reading data for city: ", city, ". remaining cities: ", totalCityNum - currentCityNum)
                # record times of calling the current currency
        
        callNum = 1
        assign("currentData", NULL, envir=.GlobalEnv)
        hadError = TRUE
        while (hadError == TRUE && callNum <= maxCallPerCity) {
            hadError = tryCatch({
                jsonList = fromJSON(linkAddress)
                assign("currentData", 
                           as.data.frame(rbind(unlist(jsonList))), 
                           envir=.GlobalEnv)
            FALSE
        }, warning = function(warn) {
                assign("currentData", NULL, envir=.GlobalEnv)
#                 message('errors')
                TRUE
            }, error = function(err) {
                assign("currentData", NULL, envir=.GlobalEnv)
#                 message('errors')
                TRUE
        })
            callNum = callNum + 1
#             message('current call number', callNum)
            Sys.sleep(callInterval)
        }
        
        if (!is.null(currentData)) {
            if (currentCityNum == 1) {
                write_csv(currentData, "data/electricity_price_OR_re.csv")
            } else {
                write_csv(currentData, "data/electricity_price_OR_re.csv", append = TRUE)
            }
            message(city, " data is successfully written to electricity_price_WY.csv")
        } else {
           message(city, " data retrieving failed")
        }
        
        currentCityNum = currentCityNum + 1
            
    }
}

### Skip next line if the data has already been retrieved.

In [9]:
get_electricity_prices(places)

reading data for city: Antelope. remaining cities: 372
Antelope data is successfully written to electricity_price_WY.csv
reading data for city: Aurora. remaining cities: 371
Aurora data is successfully written to electricity_price_WY.csv
reading data for city: Beaverton. remaining cities: 370
Beaverton data is successfully written to electricity_price_WY.csv
reading data for city: Beavercreek. remaining cities: 369
Beavercreek data is successfully written to electricity_price_WY.csv
reading data for city: Boring. remaining cities: 368
Boring data is successfully written to electricity_price_WY.csv
reading data for city: Bridal%20Veil. remaining cities: 367
Bridal%20Veil data is successfully written to electricity_price_WY.csv
reading data for city: Brightwood. remaining cities: 366
Brightwood data is successfully written to electricity_price_WY.csv
reading data for city: Canby. remaining cities: 365
Canby data is successfully written to electricity_price_WY.csv
reading data for city: C

reading data for city: Cloverdale. remaining cities: 305
Cloverdale data is successfully written to electricity_price_WY.csv
reading data for city: Cornelius. remaining cities: 304
Cornelius data is successfully written to electricity_price_WY.csv
reading data for city: Dayton. remaining cities: 303
Dayton data is successfully written to electricity_price_WY.csv
reading data for city: Dundee. remaining cities: 302
Dundee data is successfully written to electricity_price_WY.csv
reading data for city: Forest%20Grove. remaining cities: 301
Forest%20Grove data is successfully written to electricity_price_WY.csv
reading data for city: Gales%20Creek. remaining cities: 300
Gales%20Creek data is successfully written to electricity_price_WY.csv
reading data for city: Garibaldi. remaining cities: 299
Garibaldi data is successfully written to electricity_price_WY.csv
reading data for city: Gaston. remaining cities: 298
Gaston data is successfully written to electricity_price_WY.csv
reading data f

reading data for city: Otter%20Rock. remaining cities: 238
Otter%20Rock data is successfully written to electricity_price_WY.csv
reading data for city: Philomath. remaining cities: 237
Philomath data is successfully written to electricity_price_WY.csv
reading data for city: Rickreall. remaining cities: 236
Rickreall data is successfully written to electricity_price_WY.csv
reading data for city: Saint%20Benedict. remaining cities: 235
Saint%20Benedict data is successfully written to electricity_price_WY.csv
reading data for city: Scio. remaining cities: 234
Scio data is successfully written to electricity_price_WY.csv
reading data for city: Scotts%20Mills. remaining cities: 233
Scotts%20Mills data is successfully written to electricity_price_WY.csv
reading data for city: Seal%20Rock. remaining cities: 232
Seal%20Rock data is successfully written to electricity_price_WY.csv
reading data for city: Shedd. remaining cities: 231
Shedd data is successfully written to electricity_price_WY.csv


reading data for city: Noti. remaining cities: 171
Noti data is successfully written to electricity_price_WY.csv
reading data for city: Oakland. remaining cities: 170
Oakland data is successfully written to electricity_price_WY.csv
reading data for city: Oakridge. remaining cities: 169
Oakridge data is successfully written to electricity_price_WY.csv
reading data for city: Ophir. remaining cities: 168
Ophir data is successfully written to electricity_price_WY.csv
reading data for city: Port%20Orford. remaining cities: 167
Port%20Orford data is successfully written to electricity_price_WY.csv
reading data for city: Powers. remaining cities: 166
Powers data is successfully written to electricity_price_WY.csv
reading data for city: Reedsport. remaining cities: 165
Reedsport data is successfully written to electricity_price_WY.csv
reading data for city: Riddle. remaining cities: 164
Riddle data is successfully written to electricity_price_WY.csv
reading data for city: Roseburg. remaining c

New%20Pine%20Creek data is successfully written to electricity_price_WY.csv
reading data for city: Paisley. remaining cities: 103
Paisley data is successfully written to electricity_price_WY.csv
reading data for city: Plush. remaining cities: 102
Plush data is successfully written to electricity_price_WY.csv
reading data for city: Silver%20Lake. remaining cities: 101
Silver%20Lake data is successfully written to electricity_price_WY.csv
reading data for city: Sprague%20River. remaining cities: 100
Sprague%20River data is successfully written to electricity_price_WY.csv
reading data for city: Summer%20Lake. remaining cities: 99
Summer%20Lake data is successfully written to electricity_price_WY.csv
reading data for city: Christmas%20Valley. remaining cities: 98
Christmas%20Valley data is successfully written to electricity_price_WY.csv
reading data for city: Bend. remaining cities: 97
Bend data is successfully written to electricity_price_WY.csv
reading data for city: Fields. remaining c

reading data for city: Meacham. remaining cities: 35
Meacham data is successfully written to electricity_price_WY.csv
reading data for city: Mikkalo. remaining cities: 34
Mikkalo data is successfully written to electricity_price_WY.csv
reading data for city: Milton%20Freewater. remaining cities: 33
Milton%20Freewater data is successfully written to electricity_price_WY.csv
reading data for city: Monument. remaining cities: 32
Monument data is successfully written to electricity_price_WY.csv
reading data for city: Mount%20Vernon. remaining cities: 31
Mount%20Vernon data is successfully written to electricity_price_WY.csv
reading data for city: North%20Powder. remaining cities: 30
North%20Powder data is successfully written to electricity_price_WY.csv
reading data for city: Pilot%20Rock. remaining cities: 29
Pilot%20Rock data is successfully written to electricity_price_WY.csv
reading data for city: Prairie%20City. remaining cities: 28
Prairie%20City data is successfully written to elect

Read retrieved data

In [11]:
electricity.prices = read_csv("data/electricity_price_OR_re.csv")
electricity.prices

Parsed with column specification:
cols(
  inputs.address = col_character(),
  version = col_character(),
  metadata.sources = col_character(),
  outputs.company_id = col_character(),
  outputs.utility_name = col_character(),
  outputs.utility_info.company_id = col_character(),
  outputs.utility_info.utility_name = col_character(),
  outputs.commercial = col_character(),
  outputs.industrial = col_character(),
  outputs.residential = col_character()
)
Warning message in rbind(names(probs), probs_f):
“number of columns of result is not a multiple of vector length (arg 1)”Warning message:
“69 parsing failures.
row # A tibble: 5 x 5 col     row   col   expected    actual                               file expected   <int> <chr>      <chr>     <chr>                              <chr> actual 1     6  <NA> 10 columns 8 columns 'data/electricity_price_OR_re.csv' file 2    15  <NA> 10 columns 8 columns 'data/electricity_price_OR_re.csv' row 3    21  <NA> 10 columns 8 columns 'data/electricity_p

inputs.address,version,metadata.sources,outputs.company_id,outputs.utility_name,outputs.utility_info.company_id,outputs.utility_info.utility_name,outputs.commercial,outputs.industrial,outputs.residential
Antelope,3.1.0,Ventyx Research (2012),20138,Wasco Electric Coop Inc,20138,Wasco Electric Coop Inc,0.0966,0.0739,0.0956
Aurora,3.1.0,Ventyx Research (2012),4110,Commonwealth Edison Co,4110,Commonwealth Edison Co,0.0405,0.0139,0.1044
Beaverton,3.1.0,Ventyx Research (2012),15248,Portland General Electric Co,15248,Portland General Electric Co,0.0845,0.0535,0.1072
Beavercreek,3.1.0,Ventyx Research (2012),4922,Dayton Power & Light Co (The),4922,Dayton Power & Light Co (The),0.0483,0.0196,0.1179
Boring,3.1.0,Ventyx Research (2012),15248,Portland General Electric Co,15248,Portland General Electric Co,0.0845,0.0535,0.1072
Bridal Veil,3.1.0,Ventyx Research (2012),no data,no data,0.0831,0.0559,0.098,NA,NA
Brightwood,3.1.0,Ventyx Research (2012),15248,Portland General Electric Co,15248,Portland General Electric Co,0.0845,0.0535,0.1072
Canby,3.1.0,Ventyx Research (2012),2955,Canby Utility Board,2955,Canby Utility Board,0.051,0.0532,0.0641
Cascade Locks,3.1.0,Ventyx Research (2012),3136,Cascade Locks OR (City of),3136,Cascade Locks OR (City of),0.0807,0.0559,0.0878
Clackamas,3.1.0,Ventyx Research (2012),15248,Portland General Electric Co,15248,Portland General Electric Co,0.0845,0.0535,0.1072


Show problems

In [12]:
summary(as.factor(problems(electricity.prices)$actual))

12 columns  8 columns 
        27         42

The "12 columns" means the places have multiple utility company info casusing column mismatch, and the "8 columns" means no info.

In [13]:
problem.rows = problems(electricity.prices) %>%
    filter(actual == '12 columns') %>%
    .$row

Retrieve again for problem rows.

In [14]:
cities.retry = electricity.prices %>%
    filter(row_number() %in% problem.rows) %>%
    .$inputs.address

### Skip next line if the data has already been retrieved.

In [15]:
get_electricity_prices(cities.retry)

reading data for city: Cornelius. remaining cities: 26
Cornelius data is successfully written to electricity_price_WY.csv
reading data for city: Lafayette. remaining cities: 25
Lafayette data is successfully written to electricity_price_WY.csv
reading data for city: Sherwood. remaining cities: 24
Sherwood data is successfully written to electricity_price_WY.csv
reading data for city: Eddyville. remaining cities: 23
Eddyville data is successfully written to electricity_price_WY.csv
reading data for city: Azalea. remaining cities: 22
Azalea data is successfully written to electricity_price_WY.csv
reading data for city: Canyonville. remaining cities: 21
Canyonville data is successfully written to electricity_price_WY.csv
reading data for city: Days Creek. remaining cities: 20
Days Creek data retrieving failed
reading data for city: Dexter. remaining cities: 19
Dexter data is successfully written to electricity_price_WY.csv
reading data for city: Fall Creek. remaining cities: 18
Fall Creek

Read retried files

In [16]:
electricity.prices.retried = read_csv('data/electricity_price_OR_re.csv')
head(electricity.prices.retried)

Parsed with column specification:
cols(
  inputs.address = col_character(),
  version = col_character(),
  metadata.sources = col_character(),
  outputs.company_id = col_character(),
  outputs.utility_name = col_character(),
  outputs.utility_info.company_id1 = col_integer(),
  outputs.utility_info.company_id2 = col_integer(),
  outputs.utility_info.utility_name1 = col_character(),
  outputs.utility_info.utility_name2 = col_character(),
  outputs.commercial = col_double(),
  outputs.industrial = col_double(),
  outputs.residential = col_double()
)


inputs.address,version,metadata.sources,outputs.company_id,outputs.utility_name,outputs.utility_info.company_id1,outputs.utility_info.company_id2,outputs.utility_info.utility_name1,outputs.utility_info.utility_name2,outputs.commercial,outputs.industrial,outputs.residential
Cornelius,3.1.0,Ventyx Research (2012),4365|5416,Cornelius NC (City of)|Duke Energy Carolinas,4365,5416,Cornelius NC (City of),Duke Energy Carolinas,0.0882,0.0630,0.1000
Lafayette,3.1.0,Ventyx Research (2012),55936|9096,Entergy Gulf States Louisiana LLC|Lafayette Utilities System,55936,9096,Entergy Gulf States Louisiana LLC,Lafayette Utilities System,0.0753,0.0458,0.0796
Sherwood,3.1.0,Ventyx Research (2012),814|13718,Entergy Arkansas Inc|North Little Rock AR (City of),814,13718,Entergy Arkansas Inc,North Little Rock AR (City of),0.0885,0.0736,0.1018
Eddyville,3.1.0,Ventyx Research (2012),10171|18642,Kentucky Utilities Co|Tennessee Valley Authority,10171,18642,Kentucky Utilities Co,Tennessee Valley Authority,0.0841,0.0504,0.0883
Azalea,3.1.0,Ventyx Research (2012),20169|14354,Avista Corp|PacifiCorp,20169,14354,Avista Corp,PacifiCorp,0.0843,0.0675,0.1038
Canyonville,3.1.0,Ventyx Research (2012),20169|14354,Avista Corp|PacifiCorp,20169,14354,Avista Corp,PacifiCorp,0.0843,0.0675,0.1038


Remove error rows of the raw file and combine with the retried file.

In [17]:
as.numeric('no data')

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

[1] NA

In [18]:
electricity.prices.cleaned = electricity.prices %>%
    filter(!(row_number() %in% problems(electricity.prices)$row)) %>%
    mutate(outputs.commercial = as.numeric(outputs.commercial), 
           outputs.industrial = as.numeric(outputs.industrial),
           outputs.residential = as.numeric(outputs.residential))

electricity.prices = electricity.prices.cleaned %>%
    select(address=inputs.address, commercial=outputs.commercial, industrial=outputs.industrial, residential=outputs.residential,
           source=metadata.sources, version, company_id=outputs.company_id, utility_name=outputs.utility_name) %>%
    bind_rows(electricity.prices.retried %>%
                  select(address=inputs.address, commercial=outputs.commercial, industrial=outputs.industrial, residential=outputs.residential,
                         source=metadata.sources, version, company_id=outputs.company_id, utility_name=outputs.utility_name))
electricity.prices

Warning message in evalq(as.numeric(outputs.commercial), <environment>):
“NAs introduced by coercion”Warning message in evalq(as.numeric(outputs.industrial), <environment>):
“NAs introduced by coercion”Warning message in evalq(as.numeric(outputs.residential), <environment>):
“NAs introduced by coercion”

address,commercial,industrial,residential,source,version,company_id,utility_name
Antelope,0.0966,0.0739,0.0956,Ventyx Research (2012),3.1.0,20138,Wasco Electric Coop Inc
Aurora,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co
Beaverton,0.0845,0.0535,0.1072,Ventyx Research (2012),3.1.0,15248,Portland General Electric Co
Beavercreek,0.0483,0.0196,0.1179,Ventyx Research (2012),3.1.0,4922,Dayton Power & Light Co (The)
Boring,0.0845,0.0535,0.1072,Ventyx Research (2012),3.1.0,15248,Portland General Electric Co
Brightwood,0.0845,0.0535,0.1072,Ventyx Research (2012),3.1.0,15248,Portland General Electric Co
Canby,0.0510,0.0532,0.0641,Ventyx Research (2012),3.1.0,2955,Canby Utility Board
Cascade Locks,0.0807,0.0559,0.0878,Ventyx Research (2012),3.1.0,3136,Cascade Locks OR (City of)
Clackamas,0.0845,0.0535,0.1072,Ventyx Research (2012),3.1.0,15248,Portland General Electric Co
Clatskanie,0.0514,0.0426,0.0484,Ventyx Research (2012),3.1.0,28541,Clatskanie Peoples Utility District


Top 10 across two states ranked by commercial, industrial, residential

In [21]:
electricity.prices %>%
    inner_join(cities.selected, by = c("address"="Place Name")) %>%
    arrange(commercial, industrial, residential) %>%
    head(20)

address,commercial,industrial,residential,source,version,company_id,utility_name,State
Harrisburg,0.0276,0.0064,0.0820,Ventyx Research (2012),3.1.0,14715,PPL Electric Utilities Corp,Oregon
Amity,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Oregon
Gilchrist,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenCIPS,Oregon
Mount Vernon,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenIP,Oregon
Beaver,0.0372,0.0135,0.1144,Ventyx Research (2012),3.1.0,5487,Duquesne Light Co,Oregon
Aurora,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon
Lyons,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon
Silver Lake,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon
Elgin,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon
Riverside,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon


Top 10 across two states ranked by industrial, commercial, residential

In [20]:
electricity.prices %>%
    inner_join(cities.selected, by = c("address"="Place Name")) %>%
    arrange(industrial, commercial, residential) %>%
    head(20)

address,commercial,industrial,residential,source,version,company_id,utility_name,State
Harrisburg,0.0276,0.0064,0.0820,Ventyx Research (2012),3.1.0,14715,PPL Electric Utilities Corp,Oregon
Gilchrist,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenCIPS,Oregon
Mount Vernon,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenIP,Oregon
Amity,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Oregon
Beaver,0.0372,0.0135,0.1144,Ventyx Research (2012),3.1.0,5487,Duquesne Light Co,Oregon
Aurora,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon
Lyons,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon
Silver Lake,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon
Elgin,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon
Riverside,0.0405,0.0139,0.1044,Ventyx Research (2012),3.1.0,4110,Commonwealth Edison Co,Oregon


In [25]:
electricity.prices %>%
    filter(address=='Hermiston')

address,commercial,industrial,residential,source,version,company_id,utility_name
Hermiston,0.0635,0.0518,0.0739,Ventyx Research (2012),3.1.0,8515|19325,Hermiston OR (City of)|Umatilla Electric Coop


Write to file

In [26]:
electricity.prices %>%
    write_csv('data/electricity_price_OR.csv')

### Idaho

In [13]:
electricity.prices %>%
    inner_join(cities.selected, by = c("address"="Place Name")) %>%
    filter(State == 'Idaho') %>%
    distinct() %>%
    arrange(commercial, industrial, residential) %>%
    head(10)

address,commercial,industrial,residential,source,version,company_id,utility_name,State
Moreland,0.0276,0.0064,0.082,Ventyx Research (2012),3.1.0,14715,PPL Electric Utilities Corp,Idaho
Irwin,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Idaho
Blanchard,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Idaho
Lake Fork,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenCILCO,Idaho
Rockland,0.0415,0.0228,0.0694,Ventyx Research (2012),3.1.0,3266,Central Maine Power Co,Idaho
Lewiston,0.0415,0.0228,0.0694,Ventyx Research (2012),3.1.0,3266,Central Maine Power Co,Idaho
Dayton,0.0483,0.0196,0.1179,Ventyx Research (2012),3.1.0,4922,Dayton Power & Light Co (The),Idaho
Swanlake,0.0494,0.0208,0.0924,Ventyx Research (2012),3.1.0,13511,New York State Electric & Gas Corp,Idaho
Weiser,0.0544,0.0484,0.0661,Ventyx Research (2012),3.1.0,20297,Weiser ID (City of),Idaho
Burley,0.0576,0.0548,0.0677,Ventyx Research (2012),3.1.0,2545,Burley ID (City of),Idaho


In [14]:
electricity.prices %>%
    inner_join(cities.selected, by = c("address"="Place Name")) %>%
    filter(State == 'Idaho') %>%
    distinct() %>%
    arrange(industrial, commercial, residential) %>%
    head(10)

address,commercial,industrial,residential,source,version,company_id,utility_name,State
Moreland,0.0276,0.0064,0.082,Ventyx Research (2012),3.1.0,14715,PPL Electric Utilities Corp,Idaho
Lake Fork,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenCILCO,Idaho
Irwin,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Idaho
Blanchard,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Idaho
Dayton,0.0483,0.0196,0.1179,Ventyx Research (2012),3.1.0,4922,Dayton Power & Light Co (The),Idaho
Swanlake,0.0494,0.0208,0.0924,Ventyx Research (2012),3.1.0,13511,New York State Electric & Gas Corp,Idaho
Rockland,0.0415,0.0228,0.0694,Ventyx Research (2012),3.1.0,3266,Central Maine Power Co,Idaho
Lewiston,0.0415,0.0228,0.0694,Ventyx Research (2012),3.1.0,3266,Central Maine Power Co,Idaho
Springfield,0.0771,0.0328,0.0977,Ventyx Research (2012),3.1.0,17828,Springfield Water Light & Power Dept,Idaho
Challis,0.068,0.0336,0.0796,Ventyx Research (2012),3.1.0,16565,Salmon River Electric Coop Inc,Idaho


### Washington

In [27]:
electricity.prices %>%
    inner_join(cities.selected, by = c("address"="Place Name")) %>%
    filter(State == 'Washington') %>%
    filter(commercial =='no data') %>%
    nrow()

[1] 421

In [35]:
electricity.prices %>%
    inner_join(cities.selected, by = c("address"="Place Name")) %>%
    filter(State == 'Washington') %>%
    distinct() %>%
    inner_join(cities.usa %>% select(zip=`Zip Code`, address=`Place Name`) %>%
                  group_by(address) %>%
                  summarize(zip=min(zip)) %>%
                  ungroup()) %>%
    arrange(commercial, industrial, residential) %>%
    head(20)

Joining, by = "address"


address,commercial,industrial,residential,source,version,company_id,utility_name,State,zip
East Wenatchee,0.0193,0.0239,0.0264,Ventyx Research (2012),3.1.0,5326,PUD No 1 of Douglas County,Washington,98802
Orondo,0.0193,0.0239,0.0264,Ventyx Research (2012),3.1.0,5326,PUD No 1 of Douglas County,Washington,98843
Dupont,0.0276,0.0064,0.082,Ventyx Research (2012),3.1.0,14715,PPL Electric Utilities Corp,Washington,45837
Vaughn,0.0276,0.0064,0.082,Ventyx Research (2012),3.1.0,14715,PPL Electric Utilities Corp,Washington,59487
Acme,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Washington,15610
Everson,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Washington,15631
Uniontown,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Washington,15401
Wenatchee,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,3413,PUD No 1 of Chelan County,Washington,98801
Ardenvoir,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,3413,PUD No 1 of Chelan County,Washington,98811
Cashmere,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,NA,Cashmere Light Dept,Washington,98815


In [16]:
electricity.prices %>%
    inner_join(cities.selected, by = c("address"="Place Name")) %>%
    filter(State == 'Washington') %>%
    distinct() %>%
    arrange(industrial, commercial, residential) %>%
    head(20)

address,commercial,industrial,residential,source,version,company_id,utility_name,State
Dupont,0.0276,0.0064,0.082,Ventyx Research (2012),3.1.0,14715,PPL Electric Utilities Corp,Washington
Vaughn,0.0276,0.0064,0.082,Ventyx Research (2012),3.1.0,14715,PPL Electric Utilities Corp,Washington
Mount Vernon,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenIP,Washington
Morton,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenCILCO,Washington
Centralia,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenIP,Washington
Quincy,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenCIPS,Washington
Danville,0.0363,0.0077,0.0906,Ventyx Research (2012),3.1.0,NA,AmerenIP,Washington
Harrington,0.0435,0.0099,0.1365,Ventyx Research (2012),3.1.0,5027,Delmarva Power & Light Co,Washington
Acme,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Washington
Everson,0.0337,0.0112,0.0746,Ventyx Research (2012),3.1.0,20387,West Penn Power Co,Washington


In [28]:
electricity.prices %>%
    inner_join(cities.selected, by = c("address"="Place Name")) %>%
    filter(State == 'Washington', startsWith(utility_name, "PUD")) %>%
    distinct() %>%
    arrange(industrial, commercial, residential) %>%
    head(10)

address,commercial,industrial,residential,source,version,company_id,utility_name,State
Wenatchee,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,3413,PUD No 1 of Chelan County,Washington
Ardenvoir,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,3413,PUD No 1 of Chelan County,Washington
Chelan Falls,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,3413,PUD No 1 of Chelan County,Washington
Entiat,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,3413,PUD No 1 of Chelan County,Washington
Peshastin,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,3413,PUD No 1 of Chelan County,Washington
Stehekin,0.0344,0.0218,0.0322,Ventyx Research (2012),3.1.0,3413,PUD No 1 of Chelan County,Washington
East Wenatchee,0.0193,0.0239,0.0264,Ventyx Research (2012),3.1.0,5326,PUD No 1 of Douglas County,Washington
Orondo,0.0193,0.0239,0.0264,Ventyx Research (2012),3.1.0,5326,PUD No 1 of Douglas County,Washington
Moses Lake,0.0374,0.0328,0.0485,Ventyx Research (2012),3.1.0,14624,PUD No 2 of Grant County,Washington
Soap Lake,0.0374,0.0328,0.0485,Ventyx Research (2012),3.1.0,14624,PUD No 2 of Grant County,Washington


In [31]:
cities.usa %>%
    filter(endsWith(`Place Name`, "Wenatchee"))

Zip Code,Place Name,State,State Abbreviation,County,Latitude,Longitude
98801,Wenatchee,Washington,WA,Chelan,47.4253,-120.3273
98802,East Wenatchee,Washington,WA,Douglas,47.4186,-120.2731
98807,Wenatchee,Washington,WA,Chelan,47.4235,-120.3103


Define a function to calculate distance based on coordinates

In [35]:
get_distance = function(latFrom, lonFrom, latTo, lonTo) {
    return(sqrt((latTo - latFrom)^2 + (lonTo - lonFrom)^2))
}

In [41]:
cities.usa %>%
    mutate(distance = get_distance(47.4253, -120.3273, Latitude, Longitude)) %>%
    filter(distance <= 1) %>%
    arrange(distance) %>%
    head(10)

Zip Code,Place Name,State,State Abbreviation,County,Latitude,Longitude,distance
98801,Wenatchee,Washington,WA,Chelan,47.4253,-120.3273,0.00000000
98807,Wenatchee,Washington,WA,Chelan,47.4235,-120.3103,0.01709503
98802,East Wenatchee,Washington,WA,Douglas,47.4186,-120.2731,0.05461254
98836,Monitor,Washington,WA,Chelan,47.4852,-120.4158,0.10686562
98828,Malaga,Washington,WA,Chelan,47.3553,-120.2086,0.13780308
98850,Rock Island,Washington,WA,Douglas,47.3706,-120.1378,0.19723676
98815,Cashmere,Washington,WA,Chelan,47.5173,-120.5033,0.19859507
98821,Dryden,Washington,WA,Chelan,47.5412,-120.5609,0.26077149
98822,Entiat,Washington,WA,Chelan,47.7057,-120.2760,0.28505412
98847,Peshastin,Washington,WA,Chelan,47.5458,-120.5961,0.29457374
